In [560]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime as dt

In [561]:
gov_2018 = pd.read_csv('governor_polls_2018_inprogress.csv')

In [563]:
#Move from one row per candidate to one row per poll
#What's the last poll we've been working on?
oldno = -10
#What's the index of that last poll?
masterindex = 0
for index, pollno in enumerate(gov_2018['poll_id']):
    #If we've seen this poll_id before, this row deals with a second, third, etc candidate:
    if oldno == pollno:
        #If we haven't set stats for candidate 2 yet:
        if np.isnan(gov_2018['cand2_pct'][masterindex]):
            #Set them equal to this row
            gov_2018['cand2_pct'][masterindex] = gov_2018['cand1_pct'][index]
            gov_2018['cand2_name'][masterindex] = gov_2018['cand1_name'][index]
        #If we're still on this poll, there could be a third candidate, but their name isn't tracked
        elif np.isnan(gov_2018['cand3_pct'][masterindex]):
            gov_2018['cand3_pct'][masterindex] = gov_2018['cand1_pct'][index]
        #Some polls track even more candidates, but our model doesn't, so just don't do anything 
        #if there's even more rows from this poll
    else:
        #This is a new poll, so update the reference information and keep going
        oldno = pollno
        masterindex = index

C:\Users\marcu\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\marcu\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
C:\Users\marcu\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel impo

In [564]:
#Remove all superfluous rows
gov_2018 = gov_2018[~np.isnan(gov_2018['cand2_pct'])]

In [565]:
gov_2018['margin_poll'] = (gov_2018['cand1_pct'] - gov_2018['cand2_pct'])

In [567]:
#Credit to Mike Shultz for creating this
states = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'}

In [568]:
state_key = {state: abbr for abbr, state in states.items()}

In [569]:
gov_results = pd.read_csv('governor_results_2018.csv')

In [570]:
gov_results = gov_results.set_index('State').transpose()

In [571]:
#Add in results data and data analysis
#Still need to rework the rightcall code
gov_2018['location'] = gov_2018['state']
gov_2018 = gov_2018.reset_index()
for index in range(len(gov_2018)):
    state = gov_2018['state'][index]
    gov_2018['location'][index] = state_key[state]
    abbr = gov_2018['location'][index]
    gov_2018['cand1_actual'][index] = gov_results[abbr][0]
    gov_2018['cand2_actual'][index] = gov_results[abbr][1]
    gov_2018['margin_actual'] = gov_2018['cand1_actual'] - gov_2018['cand2_actual']
    gov_2018['margin_poll'] = gov_2018['cand1_pct'] - gov_2018['cand2_pct']
    gov_2018['bias'] = gov_2018['margin_poll'] - gov_2018['margin_actual']
    gov_2018['error'] = abs(gov_2018['bias'])
    #if ((gov_2018['cand1_pct'] > gov_2018['cand2_pct']) and (gov_2018['cand1_actual'] > gov_2018['cand2_actual'])) or ((gov_2018['cand1_pct'] < gov_2018['cand2_pct']) and (gov_2018['cand1_actual'] < gov_2018['cand2_actual'])):
        #gov_2018['rightcall'] = 1
    #else:
         #gov_2018['rightcall'] = 0

C:\Users\marcu\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\marcu\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\marcu\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [572]:
gov_2018

,index,poll_id,cycle,state,pollster,sample_size,population_full,methodology,office_type,start_date,...,margin_poll,election_date,cand1_actual,cand2_actual,margin_actual,error,bias,rightcall,comment,location
0,0,36742.0,2018.0,New York,Public Policy Polling,1403.0,rv,Live Phone/Online,Governor,4/7/2016,...,23.0,11/6/2018,59.6,36.2,23.4,0.4,-0.4,NaN,NaN,NY
1,2,52596.0,2018.0,Texas,JMC Analytics,575.0,lv,Automated Phone,Governor,5/19/2018,...,-12.0,11/6/2018,42.5,55.8,-13.3,1.3,1.3,NaN,NaN,TX
2,4,52597.0,2018.0,Texas,Quinnipiac University,961.0,rv,Live Phone,Governor,5/23/2018,...,-19.0,11/6/2018,42.5,55.8,-13.3,5.7,-5.7,NaN,NaN,TX
3,6,52598.0,2018.0,Texas,Quinnipiac University,1029.0,rv,Live Phone,Governor,4/12/2018,...,-9.0,11/6/2018,42.5,55.8,-13.3,4.3,4.3,NaN,NaN,TX
4,8,52599.0,2018.0,Illinois,We Ask America,600.0,lv,NaN,Governor,6/9/2018,...,9.0,11/6/2018,54.5,38.8,15.7,6.7,-6.7,NaN,NaN,IL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
506,1510,56825.0,2018.0,Florida,Trafalgar Group,1484.0,lv,Automated Phone,Governor,11/4/2018,...,-3.4,11/6/2018,49.2,49.6,-0.4,3.0,-3.0,NaN,NaN,FL
507,1512,56828.0,2018.0,Ohio,Trafalgar Group,1948.0,lv,Automated Phone,Governor,11/2/2018,...,3.6,11/6/2018,46.7,50.4,-3.7,7.3,7.3,NaN,NaN,OH
508,1514,56829.0,2018.0,Arizona,Harris Insights & Analytics,600.0,lv,IVR/Online,Governor,11/3/2018,...,-14.3,11/6/2018,41.8,56.0,-14.2,0.1,-0.1,NaN,NaN,AZ
509,1516,56830.0,2018.0,Florida,Harris Insights & Analytics,600.0,lv,IVR/Online,Governor,11/3/2018,...,2.4,11/6/2018,49.2,49.6,-0.4,2.8,2.8,NaN,NaN,FL


In [573]:
gov_2018.to_csv('governor_polls_2018_cleaned')

In [574]:
govTest = gov_2018.groupby('methodology').mean()

In [575]:
govTest['error']

methodology
Automated Phone      7.399111
IVR/Live Phone       6.556000
IVR/Online           5.050667
Live Phone           5.183469
Live Phone/Online    5.658333
Online               4.469231
Name: error, dtype: float64

In [544]:
#Bad code, do not run
#Enumerate is doing some weird things. I'm not sure about everything that's happening here.
gov_2018['location'] = gov_2018['state']
for index, state in enumerate(gov_2018['state']):
    gov_2018['location'][index] = state_key[state]
    abbr = gov_2018['location'][index]
    gov_2018['cand1_actual'][index] = gov_results[abbr][0]
    gov_2018['cand2_actual'][index] = gov_results[abbr][1]
    gov_2018['margin_actual'] = gov_2018['cand1_actual'] - gov_2018['cand2_actual']
    gov_2018['margin_poll'] = gov_2018['cand1_pct'] - gov_2018['cand2_pct']
    gov_2018['bias'] = gov_2018['margin_poll'] - gov_2018['margin_actual']
    gov_2018['error'] = abs(gov_2018['bias'])
    #if ((gov_2018['cand1_pct'] > gov_2018['cand2_pct']) and (gov_2018['cand1_actual'] > gov_2018['cand2_actual'])) or ((gov_2018['cand1_pct'] < gov_2018['cand2_pct']) and (gov_2018['cand1_actual'] < gov_2018['cand2_actual'])):
        #gov_2018['rightcall'] = 1
    #else:
         #gov_2018['rightcall'] = 0

ValueError: cannot insert level_0, already exists